## Import Library

In [10]:
from dotenv import dotenv_values
from openai import OpenAI
import pandas as pd
from pymysql import connect

## Ingest Data from Database

In [11]:
env_values = dotenv_values('.env')

connection = connect(host=env_values['DB_HOST'],
                     db=env_values['DB_DATABASE'],
                     user=env_values['DB_USERNAME'],
                     password=env_values['DB_PASSWORD'], # type: ignore
                     ) # type: ignore
cursor = connection.cursor()
cursor.execute("SELECT * FROM sales")
result = cursor.fetchall()

df_sales = pd.DataFrame(result, columns = ["transaction_id", "transaction_date", "sales_amount", "price", "product_category"])
df_sales

,transaction_id,transaction_date,sales_amount,price,product_category
0,1,2024-06-01,100.00,10.00,Electronics
1,2,2024-06-02,200.00,20.00,Groceries
2,3,2024-06-03,150.00,15.00,Clothing
3,4,2024-06-04,300.00,30.00,Home Appliances
4,5,2024-06-05,250.00,25.00,Toys
5,6,2024-06-06,400.00,40.00,Books
6,7,2024-06-07,350.00,35.00,Electronics
7,8,2024-06-08,450.00,45.00,Groceries
8,9,2024-06-09,500.00,50.00,Clothing
9,10,2024-06-10,600.00,60.00,Home Appliances


## AI Integration

In [12]:
client = OpenAI(
   api_key = env_values['OPENAI_API_KEY'],
   base_url = "https://api.naga.ac/v1"
)

def generate_analysis(prompt, model="gpt-3.5-turbo", temp=0, max_tokens=150):
   response = client.chat.completions.create(
      model=model,
      temperature=temp,
      max_tokens=max_tokens,
      messages=[
         {
            "role": "user",
            "content": prompt
         }
      ]
   )
   
   return response.choices[0].message.content

## Generate Prompt

In [13]:
prompt = f"Buatkan SQL query mengambil data dari tabel sales berdasarkan dataset {df_sales} untuk: 1. Menghitung total penjualan per kategori produk. 2. Menghitung rata-rata harga per kategori produk. 3. Menghitung total penjualan per hari. 4. Menghitung jumlah transaksi per kategori produk."
results = generate_analysis(prompt=prompt)

# df = pd.DataFrame({"questions":[prompt], "answers":[results]})
# df.to_csv("answers.csv", mode="a", header=True)

In [14]:
print(results)

Berikut adalah SQL query untuk melakukan operasi yang diminta:

1. Menghitung total penjualan per kategori produk:
```sql
SELECT product_category, SUM(sales_amount) AS total_penjualan
FROM sales
GROUP BY product_category;
```

2. Menghitung rata-rata harga per kategori produk:
```sql
SELECT product_category, AVG(price) AS rata_rata_harga
FROM sales
GROUP BY product_category;
```

3. Menghitung total penjualan per hari:
```sql
SELECT transaction_date, SUM(sales_amount) AS total_penjualan
FROM sales
GROUP BY transaction_date;
```

4. Menghitung jumlah transaksi per kategori produk:
```sql
SELECT product_category, COUNT(transaction_id) AS jumlah_transaksi
FROM sales
GROUP BY product_category;
```

Anda dapat menjalankan query-query di atas pada database Anda untuk mendapatkan hasil yang diinginkan.


Berdasarkan hasil implementasi query dari Naga AI yang telah divalidasi pada sistem manajemen database yang ada dalam folder screenshot, dapat diketahui bahwa keempat output SQL queries di atas dapat berjalan dengan benar dan menghasilkan output seperti yang diharapkan. Terdapat "AS" di dalam query yang dihasilkan yang menunjukkan bahwa AI dapat bekerja secara efektif dengan menambahkan alias untuk sebuah nama kolom yang isinya hasil dari operasi pada kolom lainnya. Dengan hasil seperti ini, AI dapat menjadi pendukung bagi data engineer untuk mnenghasilkan query yang lebih efektif.